In [ ]:
import numpy as np
tmp = np.load('WISDM.npz', allow_pickle=True)
X = tmp['X']
X = np.squeeze(X, axis = 1)
y_one_hot = tmp['y']
folds = tmp['folds']

NameError: name 'tf' is not defined

In [31]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import os
import random
import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout, Conv2D, Lambda, Input, Bidirectional, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend as K
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
import sys
from layers import Attention, SelfAttention
import keras
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
gpus = tf.config.experimental.list_physical_devices('GPU')

def model(x_train, num_labels, LSTM_units, num_conv_filters, batch_size, F, D):
    """
    The proposed model with CNN layer, LSTM RNN layer and self attention layers.
    Inputs:
    - x_train: required for creating input shape for RNN layer in Keras
    - num_labels: number of output classes (int)
    - LSTM_units: number of RNN units (int)
    - num_conv_filters: number of CNN filters (int)
    - batch_size: number of samples to be processed in each batch
    - F: the attention length (int)
    - D: the length of the output (int) 
    Returns
    - model: A Keras model
    """
    cnn_inputs = Input(shape=(x_train.shape[1], x_train.shape[2], 1), batch_size=batch_size, name='rnn_inputs')
    cnn_layer = Conv2D(num_conv_filters, kernel_size = (1, x_train.shape[2]), strides=(1, 1), padding='valid', data_format="channels_last")
    cnn_out = cnn_layer(cnn_inputs)

    sq_layer = Lambda(lambda x: K.squeeze(x, axis = 2))
    sq_layer_out = sq_layer(cnn_out)

    rnn_layer = LSTM(units=128, activation='hard_sigmoid', return_sequences=True)(sq_layer_out)
    rnn_layer = LSTM(units=64, activation='hard_sigmoid', return_sequences=True)(rnn_layer)
    rnn_layer = Dropout(0.2)(rnn_layer)
    rnn_layer = LSTM(units=64, activation='hard_sigmoid', return_sequences=True)(rnn_layer)
    rnn_layer = LSTM(units=32, activation='hard_sigmoid', return_sequences=True)(rnn_layer)
    rnn_layer_output = Dropout(0.2)(rnn_layer)
    
    encoder_output, attention_weights = SelfAttention(size=F, num_hops=D, use_penalization=False, batch_size = batch_size)(rnn_layer_output)
    dense_layer = Dense(num_labels, activation = 'sigmoid')
    dense_layer_output = dense_layer(encoder_output)

    model = Model(inputs=cnn_inputs, outputs=dense_layer_output)
    print (model.summary())

    return model

os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

EPOCH = 300
BATCH_SIZE = 64
LSTM_UNITS = 32
CNN_FILTERS = 10
NUM_LSTM_LAYERS = 1
LEARNING_RATE = 1e-4
PATIENCE = 10
SEED = 0
F = 32
D = 10



if not os.path.exists(os.path.join(SAVE_DIR)):
    os.mkdir(os.path.join(SAVE_DIR))

SEED = 0 
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

NUM_LABELS = 1
avg_acc = []
avg_recall = []
avg_f1 = []
early_stopping_epoch_list = []

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train_ = np.expand_dims(X_train, axis = 3)
X_test_ = np.expand_dims(X_test, axis = 3)

train_trailing_samples =  X_train_.shape[0]%BATCH_SIZE
test_trailing_samples =  X_test_.shape[0]%BATCH_SIZE


if train_trailing_samples!= 0:
    X_train_ = X_train_[0:-train_trailing_samples]
    y_train = y_train[0:-train_trailing_samples]
if test_trailing_samples!= 0:
    X_test_ = X_test_[0:-test_trailing_samples]
    y_test = y_test[0:-test_trailing_samples]

print (y_train.shape, y_test.shape)   

rnn_model = model(x_train = X_train_, num_labels = NUM_LABELS, LSTM_units = LSTM_UNITS, \
    num_conv_filters = CNN_FILTERS, batch_size = BATCH_SIZE, F = F, D= D)

model_filename = './models/convLSTM.h5'
callbacks = [EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True), 
             ModelCheckpoint(filepath=model_filename, monitor = 'val_accuracy', save_weights_only=True, save_best_only=True),]#, LearningRateScheduler()]

opt = optimizers.Adam(clipnorm=1.)

rnn_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

history = rnn_model.fit(X_train_, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, verbose=1, callbacks=callbacks, validation_split=0.25)


# Evaluate model and predict data on TEST 
print("******Evaluating TEST set*********")
rnn_model.load_weights(model_filename)

y_test_predict = rnn_model.predict(X_test_, batch_size = BATCH_SIZE)
y_test_predict = np.array(y_test_predict)
y_test_predict[y_test_predict>0.5]=1
y_test_predict[y_test_predict<=0.5]=0

# all_trainable_count = int(np.sum([K.count_params(p) for p in set(rnn_model.trainable_weights)]))

# MAE = metrics.mean_absolute_error(y_test, y_test_predict, sample_weight=None, multioutput='uniform_average')

acc_fold = accuracy_score(y_test, y_test_predict)
avg_acc.append(acc_fold)

# recall_fold = recall_score(y_test, y_test_predict)
# avg_recall.append(recall_fold)

# f1_fold  = f1_score(y_test, y_test_predict)
# avg_f1.append(f1_fold)


print(avg_acc)


# print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold, i))
# print('______________________________________________________')
# K.clear_session()

# ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
# ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
# ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc = np.mean(avg_f1), scale=st.sem(avg_f1))

# print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
# print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
# print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
(6144,) (1536,)
Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rnn_inputs (InputLayer)     [(64, 10, 4068, 1)]       0         
                                                                 
 conv2d_28 (Conv2D)          (64, 10, 1, 10)           40690     
                                                                 
 lambda_28 (Lambda)          (64, 10, 10)              0         
                                                                 
 lstm_84 (LSTM)              (64, 10, 128)             71168     
                                                                 
 lstm_85 (LSTM)              (64, 10, 64)              49408     
                                                                 
 dropout_42 (Dropout)        (64, 10, 64)